In [ ]:
import requests
import pandas as pd

# Define the base API URL
base_api_url = "https://berita-indo-api-next.vercel.app/api/"

# API endpoints and types
api_data = {
    "CNN News": {"type": "cnn-news/", "listType": ["nasional", "internasional", "ekonomi", "olahraga", "teknologi", "hiburan", "gaya-hidup"]},
    "CNBC News": {"type": "cnbc-news/", "listType": ["market", "news", "entrepreneur", "syariah", "tech", "lifestyle"]},
    "Republika News": {"type": "republika-news/", "listType": ["news", "nusantara", "khazanah", "islam-digest", "internasional", "ekonomi", "sepakbola", "leisure"]},
    "Tempo News": {"type": "tempo-news/", "listType": ["nasional", "bisnis", "metro", "dunia", "bola", "sport", "cantik", "tekno", "otomotif", "nusantara"]},
}

# Create an empty DataFrame to store the data
df = pd.DataFrame(columns=["title", "link", "contentSnippet", "isoDate", "image"])

# Loop through the API endpoints
for provider, endpoint_data in api_data.items():
    for news_type in endpoint_data["listType"]:
        api_url = f"{base_api_url}{endpoint_data['type']}{news_type}"

        # (1) Hit API
        response = requests.get(api_url)

        if response.status_code == 200:
            data = response.json()
            data_list = data.get('data', [])

            # (2) Jadikan DF
            # Create a DataFrame for the current API response
            api_df = pd.DataFrame(data_list)

            # (3) Transformasi data kolom ‘isoDate’ menjadi tipe data datetime
            # Transformasi kolom 'isoDate' menjadi tipe data datetime
            api_df['isoDate'] = pd.to_datetime(api_df['isoDate'])

            # Add a 'creator' column based on the provider and news_type
            api_df["creator"] = provider.replace(" ", "") + news_type.upper()

            # Add a 'categories' column with the news_type
            api_df["categories"] = news_type

            # Concatenate the current API DataFrame with the main DataFrame
            df = pd.concat([df, api_df], ignore_index=True)

df = df[["creator", "title", "link", "categories", "isoDate", "description", "image"]]
df # banyak data NaN karena dari API-nya


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           0 non-null      object
 1   link            0 non-null      object
 2   contentSnippet  0 non-null      object
 3   isoDate         0 non-null      object
 4   image           0 non-null      object
dtypes: object(5)
memory usage: 108.0+ bytes


In [19]:
# (4) AGREGASI DATA
aggregated_data = df.groupby('creator').size().reset_index(name='count').sort_values(by='count', ascending=False)
aggregated_data

KeyError: ignored